In [2]:
import tensorflow as tf

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
# pip install tensorflow

In [4]:


# 標準使用ライブラリー
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
# Suppress warnings 
import warnings
warnings.filterwarnings('ignore')
import gc
import os
import shutil


# matplotlib and seaborn for plotting
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

# 追記
import json
import datetime
import math
plt.style.use('dark_background')

# debug
#%pdb on

import pixiedust #%pixie_debugger
import os
os.environ["CUDA_VISIBLE_DEVICES"]="5"

# tfがエラーはかないため
import tensorflow as tf
physical_devices = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)


Pixiedust database opened successfully


In [5]:
# 画像サイズ
img_size = 128
channels = 3
img_shape = (img_size, img_size, channels)

# 学習の設定
batch_size = 32
epochs = 100

画像のデータジェネレータ

In [6]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

def preprocessing_function(x):
    return x / 127.5 - 1

A_datagen = ImageDataGenerator(width_shift_range=0.1,
                 height_shift_range=0.1,
                 preprocessing_function = preprocessing_function
                 )
B_datagen = ImageDataGenerator(width_shift_range=0.1,
                 height_shift_range=0.1,
                 preprocessing_function = preprocessing_function
                 )

# apple画像へのpathを渡す
A_generator = A_datagen.flow_from_directory('./apple2orange/trainA',
                                            class_mode=None,
                                            batch_size=batch_size,
                                            target_size=(img_size, img_size), 
                                            color_mode='rgb')

# orange画像へのpathを渡す
B_generator = B_datagen.flow_from_directory('./apple2orange/trainB',
                                            class_mode=None, 
                                            batch_size=batch_size,
                                            target_size=(img_size, img_size), 
                                            color_mode='rgb')

Found 995 images belonging to 1 classes.
Found 1019 images belonging to 1 classes.


モデルの作成

In [7]:
from tensorflow.keras.initializers import RandomNormal

kernel_init = RandomNormal(mean=0.0, stddev=0.02)
gamma_init = RandomNormal(mean=0.0, stddev=0.02)

In [8]:
# ! pip install tensorflow_addons 

In [9]:
import tensorflow.keras.backend as K

import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Flatten, Reshape, UpSampling2D, BatchNormalization, Conv2D, Activation, Dropout, Conv2DTranspose
from tensorflow.keras.layers import LeakyReLU, LayerNormalization, Concatenate
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam

from tensorflow_addons.layers import InstanceNormalization


patch_size = int(img_size/2**4)
patch_shape = (patch_size, patch_size, 1)


def build_generator(filters=32):
    
    model_input = Input(shape=img_shape)
    
    # Encoder
    x = Conv2D(filters, kernel_size=4, strides=2, padding="same", use_bias=False, kernel_initializer=kernel_init)(model_input)
    x = LeakyReLU(0.2)(x)
    x1 = InstanceNormalization(gamma_initializer=gamma_init)(x)
    
    x = Conv2D(filters*2, kernel_size=4, strides=2, padding="same", use_bias=False, kernel_initializer=kernel_init)(x1)
    x = LeakyReLU(0.2)(x)
    x2 = InstanceNormalization(gamma_initializer=gamma_init)(x)
    
    x = Conv2D(filters*4, kernel_size=4, strides=2, padding="same", use_bias=False, kernel_initializer=kernel_init)(x2)
    x = LeakyReLU(0.2)(x)
    x3 = InstanceNormalization()(x)  
    
    x = Conv2D(filters*8, kernel_size=4, strides=2, padding="same", use_bias=False, kernel_initializer=kernel_init)(x3)
    x = LeakyReLU(0.2)(x)
    x4 = InstanceNormalization(gamma_initializer=gamma_init)(x)

    
    # Decoder
    x = UpSampling2D((2, 2))(x4)
    x = Conv2D(filters*4, kernel_size=4, strides=1, padding="same", use_bias=False, kernel_initializer=kernel_init)(x)
    x = LeakyReLU(0.2)(x)
    x = InstanceNormalization(gamma_initializer=gamma_init)(x)  
    x = Concatenate()([x, x3])
    
    x = UpSampling2D((2, 2))(x)
    x = Conv2D(filters*2, kernel_size=4, strides=1, padding="same", use_bias=False, kernel_initializer=kernel_init)(x)
    x = LeakyReLU(0.2)(x)
    x = InstanceNormalization(gamma_initializer=gamma_init)(x)  
    x = Concatenate()([x, x2])
    
    x = UpSampling2D((2, 2))(x)
    x = Conv2D(filters, kernel_size=4, strides=1, padding="same", use_bias=False, kernel_initializer=kernel_init)(x)
    x = LeakyReLU(0.2)(x)
    x = InstanceNormalization(gamma_initializer=gamma_init)(x)  
    x = Concatenate()([x, x1])
    
    x = UpSampling2D((2, 2))(x)
    x = Conv2D(3, kernel_size=4, strides=1, padding="same", activation="tanh", use_bias=False, kernel_initializer=kernel_init)(x)

    model = Model(model_input, x)
    
    model.summary()
    
    return model


def build_discriminator(filters=64):
    
    model_input = Input(shape=img_shape)
    
    x = Conv2D(filters, kernel_size=4, strides=2, padding="same", use_bias=False, kernel_initializer=kernel_init)(model_input)
    x = LeakyReLU(0.2)(x)
    
    x = Conv2D(filters*2, kernel_size=4, strides=2, padding="same", use_bias=False, kernel_initializer=kernel_init)(x)
    x = LeakyReLU(0.2)(x)
    x = InstanceNormalization(gamma_initializer=gamma_init)(x)
    
    x = Conv2D(filters*4, kernel_size=4, strides=2, padding="same", use_bias=False, kernel_initializer=kernel_init)(x)
    x = LeakyReLU(0.2)(x)
    x = InstanceNormalization(gamma_initializer=gamma_init)(x)  
    
    x = Conv2D(filters*8, kernel_size=4, strides=2, padding="same", use_bias=False, kernel_initializer=kernel_init)(x)
    x = LeakyReLU(0.2)(x)
    x = InstanceNormalization(gamma_initializer=gamma_init)(x)
    
    x = Conv2D(1, kernel_size=4, strides=1, padding="same", use_bias=False, kernel_initializer=kernel_init)(x)
    
    model = Model(model_input, x)
    
    model.summary()
    
    return model

In [10]:
from tensorflow.keras import optimizers

optimizer = optimizers.Adam(learning_rate=0.0002, beta_1=0.5)

lambda_cycyle = 10.
lambda_id = 5.

D_A = build_discriminator()
D_B = build_discriminator()

D_A.compile(loss='mse', optimizer=optimizer, metrics=['accuracy'])
D_B.compile(loss='mse', optimizer=optimizer, metrics=['accuracy'])

G_AB = build_generator()
G_BA = build_generator()

# 入力画像のためのプレースホルダー
real_A = Input(shape=img_shape)
real_B = Input(shape=img_shape)

# 生成器一回でできるフェイク画像
fake_A = G_BA(real_B)
fake_B = G_AB(real_A)

# サイクル整合性のための再構成
cycle_A = G_BA(fake_B)
cycle_B = G_AB(fake_A)

# 同一性
same_A = G_BA(real_A)
same_B = G_AB(real_B)

# fake画像に対する識別器の出力
disc_fake_A = D_A(fake_A)
disc_fake_B = D_B(fake_B)

D_A.trainable = False
D_B.trainable = False

cycle_gan = Model(inputs=[real_A, real_B],
                 outputs=[disc_fake_A, disc_fake_B,
                         cycle_A, cycle_B,
                         same_A, same_B])

cycle_gan.compile(loss=['mse', 'mse', 'mae', 'mae', 'mae', 'mae'],
                 loss_weights=[1, 1, lambda_cycyle, lambda_cycyle, lambda_id, lambda_id],
                 optimizer=optimizer)

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 128, 128, 3)]     0         
_________________________________________________________________
conv2d (Conv2D)              (None, 64, 64, 64)        3072      
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 64, 64, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 32, 128)       131072    
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 32, 32, 128)       0         
_________________________________________________________________
instance_normalization (Inst (None, 32, 32, 128)       256       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 16, 16, 256)      

Model: "functional_7"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            [(None, 128, 128, 3) 0                                            
__________________________________________________________________________________________________
conv2d_18 (Conv2D)              (None, 64, 64, 32)   1536        input_4[0][0]                    
__________________________________________________________________________________________________
leaky_re_lu_15 (LeakyReLU)      (None, 64, 64, 32)   0           conv2d_18[0][0]                  
__________________________________________________________________________________________________
instance_normalization_13 (Inst (None, 64, 64, 32)   64          leaky_re_lu_15[0][0]             
_______________________________________________________________________________________

学習

In [11]:
disc_A_loss_hoistory = []
disc_B_loss_hoistory = []
disc_loss_hoistory = []

gen_loss_hoistory = []


def train(epochs, iterations_per_epochs, batch_size=128):
    
    valid = np.ones((batch_size,) + patch_shape)
    fake = np.zeros((batch_size,) + patch_shape)
    
    for epoch in range(epochs):
        for i in range(iterations_per_epochs):
            
            batch_A = next(A_generator)
            batch_B = next(B_generator)
            
            # generatorのミニバッチの余りの処理。もっと実装を賢くし直した方がいい
            batch_size_modified = batch_size
            if batch_A.shape[0]!=batch_B.shape[0]:
                batch_size_modified = min(batch_A.shape[0], batch_B.shape[0])
                
            batch_A = batch_A[:batch_size_modified]
            batch_B = batch_B[:batch_size_modified]
            
            fake_B = G_AB.predict(batch_A)
            fake_A = G_BA.predict(batch_B)
            
            disc_A_loss_real = D_A.train_on_batch(batch_A, valid[:batch_size_modified])
            disc_A_loss_fake = D_A.train_on_batch(fake_A, fake[:batch_size_modified])
            
            disc_A_loss = 0.5 * np.add(disc_A_loss_real, disc_A_loss_fake)
            disc_A_loss_hoistory.append(disc_A_loss)
            
            disc_B_loss_real = D_B.train_on_batch(batch_B, valid[:batch_size_modified])
            disc_B_loss_fake = D_B.train_on_batch(fake_B, fake[:batch_size_modified])
            
            disc_B_loss = 0.5 * np.add(disc_B_loss_real, disc_B_loss_fake)
            disc_B_loss_hoistory.append(disc_B_loss)
            
            d_loss = 0.5 * np.add(disc_A_loss, disc_B_loss)
            disc_loss_hoistory.append(d_loss)
            
            g_loss = cycle_gan.train_on_batch([batch_A, batch_B],
                                              [valid[:batch_size_modified], valid[:batch_size_modified],
                                               batch_A, batch_B,
                                               batch_A, batch_B])
            gen_loss_hoistory.append(g_loss)
            
            if i==0:
                print('d loss, g loss:', d_loss[0],  g_loss)

In [12]:
iterations_per_epochs = 995//batch_size

train(epochs=epochs, iterations_per_epochs=iterations_per_epochs, batch_size=batch_size)

d loss, g loss: 0.4999165259764595 [19.16208267211914, 0.9847031235694885, 0.9841373562812805, 0.5853414535522461, 0.560097336769104, 0.5866570472717285, 0.5611135959625244]
d loss, g loss: 0.2273029424250126 [11.205707550048828, 0.5486558079719543, 0.538940966129303, 0.4102889597415924, 0.2720063030719757, 0.40206781029701233, 0.25696393847465515]
d loss, g loss: 0.11476925946772099 [10.658384323120117, 0.805019736289978, 0.6553251147270203, 0.2983318567276001, 0.3251206874847412, 0.2855234742164612, 0.3071795105934143]
d loss, g loss: 0.11971727386116982 [10.623266220092773, 0.9117295742034912, 0.747999906539917, 0.2791449725627899, 0.33174487948417664, 0.2639445662498474, 0.3069829046726227]
d loss, g loss: 0.21095392107963562 [9.530851364135742, 0.544865071773529, 0.6474292278289795, 0.2658417820930481, 0.3022240102291107, 0.2550603151321411, 0.27651941776275635]
d loss, g loss: 0.18477915972471237 [8.884145736694336, 0.5769512057304382, 0.6849018335342407, 0.25045454502105713, 0.2

d loss, g loss: 0.07115549501031637 [6.386547565460205, 0.8594188690185547, 0.8559460639953613, 0.15691252052783966, 0.15939365327358246, 0.1439490020275116, 0.1576751321554184]
d loss, g loss: 0.14552415907382965 [6.169507026672363, 0.7676623463630676, 1.0109689235687256, 0.14926262199878693, 0.14706730842590332, 0.14392362534999847, 0.14159168303012848]
d loss, g loss: 0.06780981458723545 [6.5659918785095215, 0.8677035570144653, 0.9141994714736938, 0.16286064684391022, 0.16015878319740295, 0.15462073683738708, 0.15615828335285187]
d loss, g loss: 0.09343448281288147 [6.066399574279785, 0.8475950360298157, 0.8473482131958008, 0.1490599811077118, 0.14919112622737885, 0.13700225949287415, 0.14078682661056519]
d loss, g loss: 0.10892591532319784 [6.800067901611328, 0.8282220959663391, 1.0551592111587524, 0.15462252497673035, 0.18427971005439758, 0.14054974913597107, 0.16498297452926636]
d loss, g loss: 0.10376465879380703 [6.1393513679504395, 0.7949581742286682, 0.8766424655914307, 0.149

d loss, g loss: 0.09787111170589924 [6.222155570983887, 0.978054940700531, 1.0157297849655151, 0.14077584445476532, 0.1472090631723404, 0.12926001846790314, 0.14044448733329773]


学習後のモデル（`G_AB, G_BA`）にテスト画像を入れて、うまく変換できているかを確認してみる。画素値が`[-1, 1]`に規格化されていることに注意。

In [18]:
cycle_gan.save("./cycle_gan_model.h5")

In [19]:
import cv2

iii = cv2.imread("./apple2orange/testA/img/n07740461_10011.jpg")
# A_datagen(iii)
cycle_gan.predict(iii)

ValueError: in user code:

    /opt/conda/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:1462 predict_function  *
        return step_function(self, iterator)
    /opt/conda/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:1452 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    /opt/conda/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:1211 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /opt/conda/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:2585 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /opt/conda/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:2945 _call_for_each_replica
        return fn(*args, **kwargs)
    /opt/conda/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:1445 run_step  **
        outputs = mo

In [ ]:
cycle_gan.history

In [ ]:
!ls ./apple2orange/testA/img/

In [ ]:
!ls "./apple2orange/testA/img/n07740461_10011.jpg"

In [ ]:

# Model
results = G_AB.predict(next(A_generator))

In [ ]:
# cv2.imshow("img",np.array(results[0]*127+127).astype(int))
# plt.imshow(np.array(results[0]*127+127).astype(int))

In [ ]:
# cv2.imshow()

In [ ]:
type(Model)